In [9]:
%pip install pytubefix

### Youtube에서 오디오 추출하기

In [7]:
import os
from pytubefix import YouTube

def downloadYouTube(videourl, path):
    yt = YouTube(videourl)
    yt = yt.streams.filter(only_audio=True).first()
    if not os.path.exists(path):
        os.makedirs(path)
    return yt.download(path)

url = "https://www.youtube.com/watch?v=isUudT58Xfk"
audio_filename = downloadYouTube(url, "audio")

### 로컬에서 파일 가져오기

In [ ]:
audio_filename = "../../Assets/audio/2024-08-13 09-36-16.mp4"

### colab에서 파일 가져오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

audio_filename = '/content/drive/MyDrive/OH_MY_GIRL_CLOSER.mp4'

### OpenAI API 이용하기

입력할 수 있는 최대 파일 크기는 25 MB이며 입력 파일의 형식은 mp3, mp4, mpeg, mpga, m4a, wav, webm 형식을 지원합니다. 파일 크기가 크다면 pydub 등을 이용해 잘라서 사용하면 됩니다.

response_format은 text외에도 `json, verbose_json, srt, vtt` 형식을 사용할 수 있습니다.

In [ ]:
from openai import OpenAI 

OPENAI_API_KEY='sk-xxxxx'
client = OpenAI(api_key = OPENAI_API_KEY)

with open(audio_filename, "rb") as audio_file:
    transcript = client.audio.transcripitons.create(
        file = audio_file,
        model = "whisper-1",
        response_format="text",
        language="ko",
    )
print(transcript)

### Whisper Open Source 이용하기

#### ffmpeg 설치하기

\# on Ubuntu or Debian

`sudo apt update && sudo apt install ffmpeg`

\# on MacOS using Homebrew

`brew install ffmpeg`

In [ ]:
%pip install git+https://github.com/openai/whisper.git -q
%pip install torch tiktoken

In [8]:
import whisper

model = whisper.load_model("medium")
result = model.transcribe(audio_filename)
transcript = result["text"]

100%|█████████████████████████████████████| 1.42G/1.42G [00:58<00:00, 26.2MiB/s]
/opt/anaconda3/envs/whisper/lib/python3.12/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues rela

### 요약하기

In [ ]:
from string import Template

summary_prompt = Template("""
아래 내용은 사용자의 음성을 텍스트로 변환한 내용입니다.

내용을 요약해 주세요.

내용: $input
요약:
""")

In [ ]:
from openai import OpenAI

client = OpenAI(api_key = OPENAI_API_KEY)

def summary_voice(transcript):
  chat_completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[{"role": "user", "content": summary_prompt.substitute(input=transcript)}],
  )

  return chat_completion.choices[0].message.content

summary_voice(transcript)